In [1]:
import os

In [2]:
%pwd

'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float 

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training

        # Check for 'prepare_model' 
        if "prepare_model" not in self.config:
            raise KeyError("Key 'prepare_model' is missing from config.yaml!")

        prepare_model = self.config.prepare_model  
        params = self.params

        training_data = Path(training.training_data)

        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_model.base_model_path),  
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            log_dir=Path(training.log_dir),
            params_learning_rate =params.LEARNING_RATE
        )

        return training_config

In [ ]:
import os
import shutil
import tensorflow as tf
from pathlib import Path
from cnnClassifier.entity.config_entity import TrainingConfig
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from cnnClassifier.logger import logger 
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        
        """Load and compile the base model for training"""
        logger.info("Loading base model...")
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Compile the model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        logger.info("Base model loaded and compiled successfully.")


    def train_valid_generator(self):
        """Prepare training and validation data generators"""

        logger.info("Initializing training and validation data generators...")

        # Image size must match the model input
        target_img_size = (256, 256)

        # This automatically rescales and splits the dataset
        datagenerator_kwargs = dict(
            rescale=1./255,         # normalize pixel values
            validation_split=0.20   # 20% for validation
        )

        dataflow_kwargs = dict(
            target_size=target_img_size,
            batch_size=self.config.params_batch_size,
            class_mode="binary",    # Binary classification
            interpolation="bilinear",
            color_mode="rgb",       # Convert grayscale to RGB
        )

        # Validation data generator (no augmentation)
        valid_datagenerator = ImageDataGenerator(**datagenerator_kwargs)

        combined_training_dir = os.path.join("artifacts", "data_ingestion", "Combined_Training_Data")

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=combined_training_dir,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training data generator (with optional augmentation)
        if self.config.params_is_augmentation:
            train_datagenerator = ImageDataGenerator(
                rotation_range=20,
                zoom_range=0.2,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = ImageDataGenerator(**datagenerator_kwargs)

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=combined_training_dir,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        print("Training data classes:", self.train_generator.class_indices)
        print("Number of training samples:", self.train_generator.samples)
        print("Number of validation samples:", self.valid_generator.samples)

        logger.info("Data generators initialized successfully.")



    def train(self):
        """Train the model with logging, Early Stopping, and Class Weights"""
        logger.info("Starting model training...")

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Ensure log directory exists
        log_dir = Path(self.config.root_dir) / "logs"
        log_dir.mkdir(parents=True, exist_ok=True)

        combined_training_dir = os.path.join("artifacts", "data_ingestion", "Combined_Training_Data")

        stone_dir = os.path.join(combined_training_dir, "Stone")
        non_stone_dir = os.path.join(combined_training_dir, "Non-Stone")

        if not os.path.exists(stone_dir) or not os.path.exists(non_stone_dir):
            raise FileNotFoundError("One of the class folders (Stone or Non-Stone) is missing in Combined_Training_Data")

        stone_count = len(os.listdir(stone_dir))
        non_stone_count = len(os.listdir(non_stone_dir))


        # Compute class weights
        class_weights = compute_class_weight(
            class_weight="balanced",
            classes=np.array([0, 1]),
            y=[0] * non_stone_count + [1] * stone_count
        )

        class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

        # Early Stopping Callback
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True
        )

        # TensorBoard Callback
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=str(log_dir))

        # Model Checkpoint Callback to save the best model during training
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.trained_model_path,
            monitor="val_accuracy",
            save_best_only=True,
            mode="max",
            verbose=1
        )

        try:
            history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            class_weight=class_weight_dict,
            callbacks=[early_stopping, tensorboard_callback, checkpoint]
        )
            
            
            logger.info("Model training completed successfully.")

            self.save_model(
                path=self.config.trained_model_path,
                model=self.model
            )
            logger.info(f"Model saved at {self.config.trained_model_path}")

        except Exception as e:
            logger.error(f"Training failed due to: {e}")
            raise e
        
        # Plot training & validation accuracy/loss
        try:
            metrics_dir = Path(self.config.root_dir) / "metrics"
            metrics_dir.mkdir(parents=True, exist_ok=True)
            # Plot Accuracy
            plt.figure(figsize=(8, 6))
            plt.plot(history.history['accuracy'], label='Train Accuracy')
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
            plt.title('Model Accuracy')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.grid(True)
            plt.savefig(metrics_dir / 'accuracy.png')
            plt.close()

            # Plot Loss
            plt.figure(figsize=(8, 6))
            plt.plot(history.history['loss'], label='Train Loss')
            plt.plot(history.history['val_loss'], label='Validation Loss')
            plt.title('Model Loss')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True)
            plt.savefig(metrics_dir / 'loss.png')
            plt.close()

            logger.info(f"Training metrics plotted and saved to {metrics_dir}")

        except Exception as plot_err:
            logger.warning(f"Could not plot training metrics due to: {plot_err}")

        
    def save_model(self, path: Path, model: tf.keras.Model):
        """Save the trained model to a given path."""
        model.save(path)
        logger.info(f"Model saved successfully at: {path}")       
        



In [9]:
try: 
    config = ConfigurationManager()
    training_config = config.get_training_config()  
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    logger.exception(e)
    raise e

[2025-04-05 13:11:05,233: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-05 13:11:05,236: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-05 13:11:05,239: INFO: common: created directory at: artifacts]
[2025-04-05 13:11:05,239: INFO: common: created directory at: artifacts\training]
[2025-04-05 13:11:05,241: INFO: 3633928156: Loading base model...]
[2025-04-05 13:11:05,738: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
[2025-04-05 13:11:05,749: INFO: 3633928156: Base model loaded and compiled successfully.]
[2025-04-05 13:11:05,749: INFO: 3633928156: Initializing training and validation data generators...]
Found 7763 images belonging to 2 classes.
Found 31058 images belonging to 2 classes.
Training data classes: {'Non-Stone': 0, 'Stone': 1}
Number of training samples: 31058
Number of validation samples: 7763
[2025-04-05 13:11:06,839: INFO: 3633928156: Data gene